In [1]:
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np

In [2]:
face_classifier=cv2.CascadeClassifier(r'D:/Facial_expression/haarcascade_frontalface_default.xml')
model=load_model(r'D:\Facial_expression/Emotion_little_vgg.h5')

In [3]:
class_labels=['Angry','Happy','Sad','Neutral','Surprise']


In [11]:
#video_path=r'D:/Facial_expression/test.mp4'
cap=cv2.VideoCapture(0)


In [12]:
while True:
    ret,frame=cap.read()
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces=face_classifier.detectMultiScale(gray,1.3,5)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,255,0),2) 
        # cv2.rectangle(image,start_point, end_point, color, thickness)
        roi_gray=gray[y:y+h,x:x+w] #region of interst
        roi_gray=cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)
        #cv2.resize(src, dsize[, dst[, fx[, fy[, interpolation]]]])
        if np.sum([roi_gray])!=0: # if face is there 
            roi=roi_gray.astype('float')/255.0 #resizing it by dividing it by factor of 1/255
            roi=img_to_array(roi) # so that image could be understood by computer
            roi=np.expand_dims(roi,axis=0)
            preds=model.predict(roi)[0] #prediction on our face 
            label=class_labels[preds.argmax()] #in case of confusion choose maximum argument 
            label_position=(x,y)
            cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
        else:
            cv2.putText(frame,'NO FACE FOUND',(20,60),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
    cv2.imshow('Emotion Detector',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()